# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""../../../../da…","""26746426152932…","""54808455846087…","""63558194502412…","""41345124289263…",false
1.4856e19,"""../../../../da…","""10047637690341…","""15038581206643…","""63558194502412…","""71776067379681…",true
5.6458e17,"""../../../../da…","""17791865068669…","""27126376081844…","""16852096676449…","""41345124289263…",false
4.0723e18,"""../../../../da…","""10047637690341…","""16463744014976…","""13039678450133…","""41345124289263…",false
1.4677e19,"""../../../../da…","""26746426152932…","""17426427435181…","""13039678450133…","""12849998779981…",true
1.6330e19,"""../../../../da…","""17791865068669…","""15233036943594…","""15853117393562…",null,false
8.7675e18,"""../../../../da…","""17791865068669…","""98683588696465…","""13039678450133…","""12849998779981…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""../../../../da…","""13177954549326…","""13027137914491…","""FirstMortgage3…","""84883170241311…","""Rejected"""
1.4856e19,"""../../../../da…","""14950706021234…","""44692450310799…","""FirstMortgage3…","""12935343828883…","""Accepted"""
5.6458e17,"""../../../../da…","""49955515319053…","""37223898791898…","""MoneyMarketSav…","""84883170241311…","""Rejected"""
4.0723e18,"""../../../../da…","""14950706021234…","""91205805172532…","""BasicChecking""","""84883170241311…","""Rejected"""
1.4677e19,"""../../../../da…","""13177954549326…","""40299083949176…","""BasicChecking""","""50639479409936…","""Accepted"""
1.6330e19,"""../../../../da…","""49955515319053…","""15331789333573…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""../../../../da…","""49955515319053…","""12261626134715…","""BasicChecking""","""50639479409936…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""../../../../da…","""88973727667710…","""14227642751389…","""FirstMortgage3…","""10156923950410…","""Rejected"""
1.4856e19,"""../../../../da…","""14276234519374…","""10725813118939…","""FirstMortgage3…","""22629638567759…","""Accepted"""
5.6458e17,"""../../../../da…","""88346356441639…","""61762329530913…","""MoneyMarketSav…","""10156923950410…","""Rejected"""
4.0723e18,"""../../../../da…","""14276234519374…","""15570628256164…","""BasicChecking""","""10156923950410…","""Rejected"""
1.4677e19,"""../../../../da…","""88973727667710…","""18272524013981…","""BasicChecking""","""13647358944967…","""Accepted"""
1.6330e19,"""../../../../da…","""88346356441639…","""15510860129512…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""../../../../da…","""88346356441639…","""14560138365070…","""BasicChecking""","""13647358944967…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""12679606537431…","""31633422295332…","""FirstMortgage3…","""15070299082038…","""Rejected"""
1.4856e19,"""79885302057955…","""16111234525414…","""FirstMortgage3…","""14922665906292…","""Accepted"""
5.6458e17,"""77395737721699…","""14359874167209…","""MoneyMarketSav…","""15070299082038…","""Rejected"""
4.0723e18,"""79885302057955…","""15125128049845…","""BasicChecking""","""15070299082038…","""Rejected"""
1.4677e19,"""12679606537431…","""13850579808013…","""BasicChecking""","""14380908482305…","""Accepted"""
1.6330e19,"""77395737721699…","""18167412464757…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""77395737721699…","""97114270554387…","""BasicChecking""","""14380908482305…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'